In [ ]:
import numpy as np
from src.perceptron import Perceptron
from src.trainer import TrainerConfig, train_perceptron, evaluate_perceptron
from src.readers import read_csv
from src.plotter import plot_arrays, plot_bars

In [ ]:
dataset, dataset_outputs = read_csv('TP3-ej2-conjunto.csv')

config = TrainerConfig.from_file("ejercicio2-lineal-config.json")

In [ ]:
perceptron_lin = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)

print(f"Initial weights for LINEAR perceptron: {perceptron_lin.w}")

error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_lin = train_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron after training for {result_lin.epoch_num} epoch{'' if result_lin.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_lin.end_reason.value}")
print(f"Final weights: {perceptron_lin.w}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

perceptron_tanh = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)
outputs_tanh = config.scaler.scale(dataset_outputs)

print(f"Initial weights for TANH perceptron: {perceptron_tanh.w}")

error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_tanh = train_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron after training for {result_tanh.epoch_num} epoch{'' if result_tanh.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_tanh.end_reason.value}")
print(f"Final weights: {perceptron_tanh.w}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-logistic-config.json")

perceptron_logis = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)
outputs_logis = config.scaler.scale(dataset_outputs)

print(f"Initial weights for LOGISTIC perceptron: {perceptron_logis.w}")

error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_logis = train_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron after training for {result_logis.epoch_num} epoch{'' if result_logis.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_logis.end_reason.value}")
print(f"Final weights: {perceptron_logis.w}")

In [ ]:
def map_to_percentage(array, a, b):
    range_length = b - a
    return [(value - a) / range_length for value in array]

lin_range = (0.0, 88.0)
tanh_range = (-1.0, 1.0)
logis_range = (0.0, 1.0)

erros_lin_percentage = map_to_percentage(result_lin.error_history, *lin_range)
erros_tanh_percentage = map_to_percentage(result_tanh.error_history, *tanh_range)
erros_logis_percentage = map_to_percentage(result_logis.error_history, *logis_range)
errors = [erros_lin_percentage, erros_tanh_percentage, erros_logis_percentage]
errors_labels = ["LINEAR", "TANH", "LOGISTIC"]

plot_arrays(errors, errors_labels, "Average Percentual Error per Epoch", "Epoch", "Error").plot()

In [ ]:
from src.dataset_selectors import split_random_by_percentage
from src.runner import run_n_times

In [ ]:
def get_dataset_10_training_90_testing():
    return split_random_by_percentage(dataset, dataset_outputs, 0.1)

config = TrainerConfig.from_file("ejercicio2-lineal-config.json")

(
    training_error_mean,
    training_error_stdev,
    test_error_mean,
    test_error_stdev,
) = run_n_times(
    10,
    lambda: get_dataset_10_training_90_testing(),
    config,
)

plot_bars(
    [training_error_mean, test_error_mean],
    [training_error_stdev, test_error_stdev],
    ["Training (Aprendizaje)", "Test (Generalizacion)"],
    "LINEAR - 10% Training 90% Test - Error Promedio",
    "Dataset",
    "Error",
).plot()

In [ ]:
def get_dataset_90_training_10_testing():
    return split_random_by_percentage(dataset, dataset_outputs, 0.9)

config = TrainerConfig.from_file("ejercicio2-lineal-config.json")

(
    training_error_mean,
    training_error_stdev,
    test_error_mean,
    test_error_stdev,
) = run_n_times(
    10,
    lambda: get_dataset_90_training_10_testing(),
    config,
)

plot_bars(
    [training_error_mean, test_error_mean],
    [training_error_stdev, test_error_stdev],
    ["Training (Aprendizaje)", "Test (Generalizacion)"],
    "LINEAR - 90% Training 10% Test - Error Promedio",
    "Dataset",
    "Error",
).plot()

In [ ]:
def get_dataset_90_training_10_testing():
    train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs = split_random_by_percentage(dataset, dataset_outputs, 0.9)
    train_dataset_outputs = config.scaler.scale(train_dataset_outputs)
    test_dataset_outputs = config.scaler.scale(test_dataset_outputs)
    return train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs

config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

(
    training_error_mean,
    training_error_stdev,
    test_error_mean,
    test_error_stdev,
) = run_n_times(
    10,
    lambda: get_dataset_90_training_10_testing(),
    config,
)

plot_bars(
    [training_error_mean, test_error_mean],
    [training_error_stdev, test_error_stdev],
    ["Training (Aprendizaje)", "Test (Generalizacion)"],
    "TANH - 90% Training 10% Test - Error Promedio",
    "Dataset",
    "Error",
).plot()

In [ ]:
def get_dataset_10_training_90_testing():
    train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs = split_random_by_percentage(dataset, dataset_outputs, 0.1)
    train_dataset_outputs = config.scaler.scale(train_dataset_outputs)
    test_dataset_outputs = config.scaler.scale(test_dataset_outputs)
    return train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs

config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

(
    training_error_mean,
    training_error_stdev,
    test_error_mean,
    test_error_stdev,
) = run_n_times(
    10,
    lambda: get_dataset_10_training_90_testing(),
    config,
)

plot_bars(
    [training_error_mean, test_error_mean],
    [training_error_stdev, test_error_stdev],
    ["Training (Aprendizaje)", "Test (Generalizacion)"],
    "TANH - 10% Training 90% Test - Error Promedio",
    "Dataset",
    "Error",
).plot()

In [ ]:
def get_dataset_50_training_50_testing():
    train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs = split_random_by_percentage(dataset, dataset_outputs, 0.5)
    train_dataset_outputs = config.scaler.scale(train_dataset_outputs)
    test_dataset_outputs = config.scaler.scale(test_dataset_outputs)
    return train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs

config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

(
    training_error_mean,
    training_error_stdev,
    test_error_mean,
    test_error_stdev,
) = run_n_times(
    10,
    lambda: get_dataset_50_training_50_testing(),
    config,
)

plot_bars(
    [training_error_mean, test_error_mean],
    [training_error_stdev, test_error_stdev],
    ["Training (Aprendizaje)", "Test (Generalizacion)"],
    "TANH - 50% Training 50% Test - Error Promedio",
    "Dataset",
    "Error",
).plot()